In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats
from ranking_utils import scores_better, rankscore_avgtie, greaters, r_names
import pandas as pd


results_dir = Path(Path.cwd(),'L2R_Continous_NPY') ##
if not results_dir.exists():
    ##load and extract data
    !wget https://cloud.imi.uni-luebeck.de/s/gYmJx2nms36xzB4/download -O L2R_Continous_NPY.zip
    
    !unzip L2R_Continous_NPY.zip
    
num_labels = 9
num_cases = 8
task_name = 'AbdomenMRCT'


# num_labels = 35
# num_cases = 38
# task_name = 'OASIS' ##discard 2020_MICCAI results

discard_events = [] #eg. ['2020_MICCAI', '2022_2023_Continous']
discard_teams = ['2020_MICCAI/initial',
                 '2020_MICCAI/stephanie_haeger_mevis_post',
                 '2020_MICCAI/tony_mok_hkust',
                 '2020_MICCAI/tony_mok_hkust_unsupervised',
                 '2021_MICCAI/LapIRN(unsupervised)',
                 ] #eg. ['2020_MICCAI/Initial', '2022_2023_Continous/Initial']

initial_results = pd.read_csv(results_dir / 'AbdomenMRCT_initial.csv', index_col=0)

                 

In [2]:

results_paths = sorted([x.parent for x in results_dir.glob(f'*/*/{task_name}_dice.npy') if x.is_file()])

###apply filters
results_paths = list(filter(lambda x: x.parent.name not in discard_events and x.parent.name+'/'+x.name not in discard_teams, results_paths))


#print(*results_paths, sep = "\n")
events, unique_teams = [x.parent.name for x in results_paths],[x.name for x in results_paths]

N = len(unique_teams)






In [3]:
dice_3 = np.zeros((N,num_cases,num_labels))
hd95_3 = 5*np.ones((N,num_cases,num_labels))
sdlogj_3 = 5*np.ones((N,num_cases))
times_3 = 100*np.ones((N,num_cases))


for i, r_path in enumerate(results_paths):
    team = r_names(r_path.name)
    if Path ( r_path / f'{task_name}_dice.npy').is_file():
        dice = np.load(r_path / f'{task_name}_dice.npy')
        dice[dice != dice] = 0
        dice_3[i,:,:] = dice
    if Path ( r_path / f'{task_name}_hd95.npy').is_file():
        hd95 = np.load(r_path / f'{task_name}_hd95.npy')
        hd95[hd95 != hd95] = 10
        hd95[hd95 == np.Inf] = 10
        hd95_3[i,:,:] = hd95
    if Path ( r_path / f'{task_name}_sd_log_j.npy').is_file() and not team == 'Initial':
        sdlogj = np.load(r_path / f'{task_name}_sd_log_j.npy')
        sdlogj_3[i,:] = sdlogj
    else:
        sdlogj_3[i,:] = 1.5
        
    if Path ( r_path / f'{task_name}_docker_runtimes.npy').is_file() and not team == 'Initial':
        times = np.load(r_path / f'{task_name}_docker_runtimes.npy')
        times_3[i,:] =  times
    else:
        times_3[i,:] = 5000

    # if len (team) < 5:
    #     tab = '\t\t\t\t'
    # elif len (team) < 12:
    #     tab = '\t\t\t'
    # elif len (team) < 24:
    #     tab = '\t\t'
    # else:
    #     tab = '\t'

    # print(i, team, tab + '%0.3f' % dice.mean(), '%0.3f' % hd95.mean(), '%0.3f' % sdlogj_3[i,:].mean(),
    #       '%0.3f' % times_3[i,:].mean())


    


In [4]:
#robustify
dice0 = dice_3[0]  
dice3_30 = np.zeros((N, round((num_cases*num_labels)*.3)))
idx30 = np.argsort(dice0.reshape(-1))[:dice3_30.shape[1]]
for i in range(N):
    dice3_30[i] = dice_3.reshape(N,-1)[i,idx30]




In [5]:
#robustify
dice30 = np.zeros((N, round((num_cases*num_labels)*.3)))
ordered_diff = np.loadtxt(results_dir / 'AbdomenMRCT_ordered_difficulty.csv', delimiter=",", dtype=int)

for i in range(N):
    dice3_30[i] = dice_3.reshape(N,-1)[i,ordered_diff[:round((num_cases*num_labels)*.3)]]
print(dice3_30.shape)


(13, 22)


In [6]:
rank_all = np.zeros((N,5))
subset = dice_3.reshape(N,-1)
scores = scores_better(subset, N)
rank_dice3a = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,0] = rank_dice3a
subset = dice3_30.reshape(N,-1)
scores = scores_better(subset, N)
rank_dice3b = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,1] = rank_dice3b
subset = hd95_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_hd3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,2] = rank_hd3
subset = sdlogj_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_jac3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,3] = rank_jac3
subset = times_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_time3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,4] = rank_time3

In [7]:
rank3 = np.power(np.prod(rank_all[:,:],axis=1),.2)
all_rank = np.power(np.prod(rank_all,axis=1),1/5)

In [8]:
df = pd.DataFrame(columns=['num', 'team', 'Dice', 'Dice_30', 'HD95', 'sdLogJ', 'rank'])
rank3_withoutRUNTIME = np.power(np.prod(rank_all[:,:4],axis=1),.25)
all_rank_withoutRUNTIME = np.power(np.prod(rank_all[:,:4],axis=1),1/4)
idx2 = np.argsort(-all_rank_withoutRUNTIME)


print('WITHOUT RUNTIME')
print('#','team','\t\t\t','Dice\t','Dice30\t','HD95\t',' sdLogJ','rank')


for i in range(N):
    df.loc[i,'rank'] = all_rank_withoutRUNTIME[i]
    team = r_names(unique_teams[idx2[i]])
    new = ''
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if(len(team)>=d0):
        tab = '\t\t'
    if(len(team)>d0+d1):
        tab = '\t'
    
    if(len(team)>=d0+d1*2):
        tab = ''
    #if(len(team)>=d0+d1*3):
    #    tab = ''
    print(i+1,new + team,tab,'%0.3f'%dice_3[idx2[i]].mean(),'\t','%0.3f'%dice3_30[idx2[i]].mean(),'\t','%0.3f'%hd95_3[idx2[i]].mean(),' ','%0.3f'%sdlogj_3[idx2[i]].mean(),' ','%0.3f'%rank3_withoutRUNTIME[idx2[i]])
    df.loc[i]= [i+1, team, dice_3[idx2[i]].mean(), dice3_30[idx2[i]].mean(), hd95_3[idx2[i]].mean(), sdlogj_3[idx2[i]].mean(), rank3_withoutRUNTIME[idx2[i]]]
print(i+2, 'Initial', '\t\t', '%0.3f' % float(initial_results['13']['Dice']), '\t', '%0.3f' % float(initial_results['13']['Dice_30']), '\t', '%0.3f' % float(initial_results['13']['HD95']))
df.loc[i+1]= [i+2, 'Initial', float(initial_results['13']['Dice']), float(initial_results['13']['Dice_30']), float(initial_results['13']['HD95']), '', '']


WITHOUT RUNTIME
# team 			 Dice	 Dice30	 HD95	  sdLogJ rank
1 ConvexAdam 		 0.731 	 0.612 	 17.003   0.092   0.848
2 PIMed 		 0.682 	 0.513 	 14.070   0.072   0.848
3 corrField 		 0.729 	 0.611 	 15.435   0.096   0.803
4 LapIRN 		 0.688 	 0.496 	 14.894   0.123   0.710
5 MEVIS 		 0.653 	 0.533 	 20.021   0.146   0.710
6 > LapIRNv2 		 0.731 	 0.584 	 14.604   0.131   0.710
7 NiftyReg 		 0.552 	 0.462 	 25.177   0.121   0.556
8 Driver 		 0.547 	 0.344 	 19.105   0.132   0.502
9 LaTIM 		 0.491 	 0.239 	 33.251   0.128   0.362
10 Imperial 		 0.406 	 0.246 	 40.679   0.107   0.356
11 Multi-Brain 		 0.442 	 0.266 	 30.289   0.481   0.257
12 Winter 		 0.410 	 0.200 	 27.595   0.851   0.213
13 songx 		 0.342 	 0.095 	 35.671   0.422   0.121
14 Initial 		 0.223 	 0.005 	 40.734


In [9]:
idx2 = np.argsort(-all_rank)
print('WITH RUNTIME')
print('#','team','\t\t\t','Dice\t','Dice30\t','HD95\t ','sdLogJ ','runtime\t','rank')
for i in range(N):
    team = r_names(unique_teams[idx2[i]])
    new = ''
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if(len(team)>=d0):
        tab = '\t\t'
    if(len(team)>d0+d1):
        tab = '\t'
    if(len(team)>=d0+d1*2):
        tab = ''
    #if(len(team)>=d0+d1*3):
    #    tab = ''
    print(i+1,new+team,tab,'%0.3f'%dice_3[idx2[i]].mean(),'\t','%0.3f'%dice3_30[idx2[i]].mean(),'\t','%0.3f'%hd95_3[idx2[i]].mean(),' ','%0.3f'%sdlogj_3[idx2[i]].mean(),' ','%0.4f'%times_3[idx2[i]].mean(),'\t','%0.3f'%rank3_withoutRUNTIME[idx2[i]])
print(i+2, 'Initial', '\t\t', '%0.3f' % float(initial_results['13']['Dice']), '\t', '%0.3f' % float(initial_results['13']['Dice_30']), '\t', '%0.3f' % float(initial_results['13']['HD95']))


WITH RUNTIME
# team 			 Dice	 Dice30	 HD95	  sdLogJ  runtime	 rank
1 ConvexAdam 		 0.731 	 0.612 	 17.003   0.092   1.2996 	 0.848
2 corrField 		 0.729 	 0.611 	 15.435   0.096   2.1261 	 0.803
3 LapIRN 		 0.688 	 0.496 	 14.894   0.123   1.4978 	 0.710
4 PIMed 		 0.682 	 0.513 	 14.070   0.072   59.2212 	 0.848
5 MEVIS 		 0.653 	 0.533 	 20.021   0.146   14.7296 	 0.710
6 NiftyReg 		 0.552 	 0.462 	 25.177   0.121   11.0000 	 0.556
7 > LapIRNv2 		 0.731 	 0.584 	 14.604   0.131   5000.0000 	 0.710
8 Driver 		 0.547 	 0.344 	 19.105   0.132   1.9474 	 0.502
9 Imperial 		 0.406 	 0.246 	 40.679   0.107   278.3830 	 0.356
10 LaTIM 		 0.491 	 0.239 	 33.251   0.128   5000.0000 	 0.362
11 Winter 		 0.410 	 0.200 	 27.595   0.851   2.7915 	 0.213
12 Multi-Brain 		 0.442 	 0.266 	 30.289   0.481   5000.0000 	 0.257
13 songx 		 0.342 	 0.095 	 35.671   0.422   5000.0000 	 0.121
14 Initial 		 0.223 	 0.005 	 40.734


In [10]:
print(df)

df.loc[13].drop('num').to_csv(results_dir/'AbdomenMRCT_ranking.csv',index=True)

df.to_csv('AbdomenMRCT_ranking.csv', index=False)

   num         team      Dice   Dice_30       HD95    sdLogJ      rank
0    1   ConvexAdam  0.730605  0.612326  17.003147  0.091759  0.847652
1    2        PIMed  0.681882   0.51338  14.070384  0.071944  0.847652
2    3    corrField  0.729215   0.61097  15.434509  0.095889  0.802958
3    4       LapIRN  0.687759  0.496172  14.894119  0.122943  0.710462
4    5        MEVIS   0.65322  0.532843  20.020743  0.145631  0.710462
5    6     LapIRNv2  0.731448  0.584447  14.604346  0.131021  0.710462
6    7     NiftyReg  0.552176  0.461988  25.176542  0.120981  0.556012
7    8       Driver  0.547029   0.34393  19.105023  0.131773  0.502182
8    9        LaTIM  0.490671  0.239267  33.251451   0.12812  0.362024
9   10     Imperial  0.405825  0.246194  40.679461   0.10669  0.355737
10  11  Multi-Brain  0.442374   0.26622  30.288926  0.480867  0.257421
11  12       Winter  0.409589  0.199619   27.59467  0.850929  0.213208
12  13        songx  0.342227  0.094748  35.670793  0.422141  0.120737
13  14